---------
title: Simulation study results
jupyter: python3
---------


In [ ]:
from config import *
import json
import pandas as pd
import plotnine as pn

In [ ]:
list_posteriors = []
posteriors_sims = []
failed_sims = []
for i in range(NUM_SIMS):
    try:
        f = os.path.join(RESULTS_DIR, f"posterior_{i}.npy")
        list_posteriors.append(np.load(f, allow_pickle=True))
    except Exception as e:
        print(f"Failed loading {i}: {e}")
        failed_sims.append(i)
    else:
        posteriors_sims.append(i)
posteriors = np.stack(list_posteriors)
del list_posteriors

with open(os.path.join(RESULTS_DIR, "sim_params.json"), "r") as fp:
    true_params = json.load(fp)
log_posterior = PROBABILISTIC_MODEL_CLASS(model, None, None, PRIORS)

In [ ]:
thinned_params = range(300_000, posteriors.shape[1], 100)
num_sims = posteriors.shape[0]
num_iters = posteriors.shape[1]
# df = pd.DataFrame(
#     posteriors[:,thinned_params,:].flatten(),
#     index=pd.MultiIndex.from_product([range(num_sims), thinned_params, log_posterior.get_param_names()],
#                                      names=["sim", "iteration", "param"]),
#     columns=["value"]
# ).reset_index()
df = pd.DataFrame(
    posteriors[:,thinned_params,:].flatten(),
    index=pd.MultiIndex.from_product([posteriors_sims, thinned_params, log_posterior.get_param_names()],
                                     names=["sim", "iteration", "param"]),
    columns=["value"]
).reset_index()
#del posteriors

true_vals = []
for s in posteriors_sims:
    true_vals.extend(log_posterior.param_value_dict_to_array(true_params[s]))
df_params = pd.DataFrame(
    true_vals,
    index=pd.MultiIndex.from_product([posteriors_sims, log_posterior.get_param_names()], names=["sim", "param"]),
    columns=["true"]
).reset_index()

In [ ]:
# From: https://gist.github.com/paulgb/6627336
def binom_interval(success, total, confint=0.95):
    quantile = (1 - confint) / 2.
    lower = stats.beta.ppf(quantile, success, total - success + 1)
    lower[success == 0] = 0
    upper = stats.beta.ppf(1 - quantile, success + 1, total - success)
    upper[success == total] = 1
    return (lower, upper)

"""Calculate the coverage of credible intervals.

Params:
    truth: dataframe with true values in a column labelled 'true' which is merged with the posterior
    posterior: array of posterior values
    remaining_axis_names: names of the axis of the posterior after `axis_to_remove` is removed. Should include the column containing the simulation number.
    remaining_axis_labels: labels for each cell of the axis in `remaining_axis_names`
    groups: list of column(s) the return values should be grouped by (eg: date and/or age)
    axis_to_remove: number of the axis of the posterior array which will have quantiles taken over (normally contains the samples)
    intervals: width of credible intervals to consider
    
Returns:
    dataframe with the following columns:
        those listed in `groups`
        interval: the size of the credible interval the row refers to
        sum: the number of credible intervals which contains the true value
        size: the number of credible intervals
        prop: the proportion of credible intervals which contain the true value
        ul, ll: 95% Binomail confidence interval limits for `prop` based on sample size `size`
        within: if the nominal coverage is within the confidence intervals
"""
def calc_coverage(truth, posterior, remaining_axis_names, remaining_axis_labels, groups, axis_to_remove=1, intervals = np.array([50, 75, 95])):
    lower_limits = (1 - intervals/100) / 2
    lls = []
    uls = []
    for limit in lower_limits:
        lls.append(np.quantile(posterior, limit, axis=axis_to_remove))
        uls.append(np.quantile(posterior, 1-limit, axis=axis_to_remove))
    df_intervals = pd.DataFrame(
        {"ll": np.stack(lls).flatten(), "ul": np.stack(uls).flatten()},
        index=pd.MultiIndex.from_product(
            [intervals] + remaining_axis_labels,
            names=["interval"] + remaining_axis_names
        )
    ).reset_index().merge(truth)
    df_intervals["covered"] = np.logical_and(df_intervals["ll"] < df_intervals["true"], df_intervals["true"] < df_intervals["ul"])
    coverage = df_intervals.groupby(groups + ["interval"])["covered"].agg(["sum", "size"]).reset_index()
    coverage["prop"] = coverage["sum"] / coverage["size"]
    coverage["ll"], coverage["ul"] = binom_interval(coverage["sum"], coverage["size"])
    coverage["within"] = np.logical_and(coverage["interval"]/100 > coverage["ll"], coverage["interval"]/100 < coverage["ul"])
    return coverage

In [ ]:
thinned_posterior = posteriors[:,thinned_params,:]
coverage_param = calc_coverage(df_params, thinned_posterior, ["sim", "param"], [posteriors_sims, log_posterior.get_param_names()], ["param"])

(pn.ggplot(coverage_param, pn.aes(colour="within")) +
 pn.geom_point(pn.aes(y="param", x="prop", xmin="ll", xmax="ul")) +
 pn.geom_errorbarh(pn.aes(y="param", xmin="ll", xmax="ul"), height = 0.001) +
 pn.facet_wrap("~interval") +
 pn.geom_vline(pn.aes(xintercept="interval/100"))
)

In [ ]:
(
    pn.ggplot(df[df["sim"] < 5]) +
    pn.geom_line(pn.aes("iteration", "value")) +
    pn.facet_grid("param~sim", scales="free_y") +
    pn.theme(figure_size=(7, 13))
)

In [ ]:
df_summary = pd.DataFrame(
    {"posterior_mean": thinned_posterior.mean(axis=1).flatten()},
    index=pd.MultiIndex.from_product([posteriors_sims, log_posterior.get_param_names()], names=["sim", "param"])
)\
    .reset_index()\
    .merge(df_params)
df_summary["bias"] = df_summary["posterior_mean"] - df_summary["true"]
df_summary.groupby("param")\
    .apply(lambda df: pd.DataFrame([[(df["bias"].values**2).sum(), (df["bias"].values / df["true"].values).mean()]], columns=["MSE", "Relative bias"]))

In [ ]:
dfs_ess = []
for i in posteriors_sims:
    df_ = pd.read_csv(os.path.join(RESULTS_DIR, f"ESS_{i}.npy"), names=["param", "ess"])
    df_["sim"] = i
    dfs_ess.append(df_)
df_ess = pd.concat(dfs_ess)
del dfs_ess

In [ ]:
df_ess.groupby("param")\
    .aggregate({
        "ess": ["mean", "min", "max"]
    })

In [ ]:
df_ess[(df_ess["ess"] < 100) & (df_ess["param"] != "beta[0]")]\
    .groupby("sim").count()

So sim 8 is bad but rest should be OK.


In [ ]:
df_ess[df_ess["sim"] != 8].groupby("param")\
    .aggregate({
        "ess": ["mean", "median", "min", "max"]
    })

Seems so.

# Incidence and prevalence


In [ ]:
true_results = read_csv(
    os.path.join(RESULTS_DIR, "sim_output.csv"),
    dtype={
        "age": "string"
    },
    parse_dates=[2],
    index_col=0
)
true_results["day"] = (true_results["date"] - pd.to_datetime(START_DATE)).dt.days

In [ ]:
thinned_for_predictive = range(0, thinned_posterior.shape[1], 4)
predicted_incidence = np.empty((num_sims, len(thinned_for_predictive), N_DAYS, N_STRATA))
predicted_prevalence = np.empty_like(predicted_incidence)
for i, sim in enumerate(thinned_posterior):
    for j, iter_num in enumerate(thinned_for_predictive):
        result = log_posterior.simulate(sim[iter_num])[0]
        predicted_incidence[i, j] = result[0]
        predicted_prevalence[i, j] = result[1]

In [ ]:
incidence_coverage = calc_coverage(
    truth=true_results.rename(columns={"incidence": "true"}),
    posterior=predicted_incidence,
    remaining_axis_names=["sim", "day", "age"],
    remaining_axis_labels=[posteriors_sims, range(N_DAYS), STRATA_NAMES],
    groups=["day", "age"]
)
prevalence_coverage = calc_coverage(
    truth=true_results.rename(columns={"prevalence": "true"}),
    posterior=predicted_prevalence,
    remaining_axis_names=["sim", "day", "age"],
    remaining_axis_labels=[posteriors_sims, range(N_DAYS), STRATA_NAMES],
    groups=["day", "age"]
)

In [ ]:
(
    pn.ggplot(incidence_coverage) +
    pn.geom_line(pn.aes("day", "prop", colour="factor(interval)")) +
    pn.geom_ribbon(pn.aes("day", ymin="ll", ymax="ul", fill="factor(interval)"), alpha = 0.5) +
    pn.facet_wrap("~age") +
    pn.geom_hline(pn.aes(yintercept="interval/100"))
)

In [ ]:
(
    pn.ggplot(prevalence_coverage) +
    pn.geom_line(pn.aes("day", "prop", colour="factor(interval)")) +
    pn.geom_ribbon(pn.aes("day", ymin="ll", ymax="ul", fill="factor(interval)"), alpha = 0.5) +
    pn.facet_wrap("~age") +
    pn.geom_hline(pn.aes(yintercept="interval/100"))
)

Check for bias in this


In [ ]:
posterior_predictive = pd.DataFrame(
    {
        "incidence": predicted_incidence.flatten(),
        "prevalence": predicted_prevalence.flatten(),
    },
    index=pd.MultiIndex.from_product(
        [posteriors_sims, thinned_for_predictive, range(N_DAYS), STRATA_NAMES],
        names=["sim", "iteration", "day", "age"]
    )
)
compare_predictives = posterior_predictive\
    .groupby(["sim", "day", "age"])\
    .agg({
        "incidence": "mean",
        "prevalence": "mean",
    })\
    .reset_index()\
    .merge(
        true_results,
        on=["sim", "day", "age"],
        suffixes=("_mean", "_true")
    )
compare_predictives["incidence_rel_error"] = (
    (compare_predictives["incidence_mean"] - compare_predictives["incidence_true"])
        / compare_predictives["incidence_mean"]
)
compare_predictives["prevalence_rel_error"] = (
    (compare_predictives["prevalence_mean"] - compare_predictives["prevalence_true"])
        / compare_predictives["prevalence_mean"]
)
compare_predictives = compare_predictives\
    .groupby(["day", "age"])\
    .agg({
        "incidence_rel_error": "mean",
        "prevalence_rel_error": "mean",
    })\
    .reset_index()

In [ ]:
(
    pn.ggplot(compare_predictives) +
    pn.geom_line(pn.aes("day", "prevalence_rel_error")) +
    pn.facet_wrap("~age")
)

In [ ]:
(
    pn.ggplot(compare_predictives) +
    pn.geom_line(pn.aes("day", "incidence_rel_error")) +
    pn.facet_wrap("~age")
)

In [ ]:
posterior_summary = posterior_predictive\
    .reset_index()\
    .groupby(["sim", "day", "age"])\
    .aggregate({
        "incidence": "mean",
        "prevalence": "mean",
    })\
    .reset_index()
(
    pn.ggplot(
        posterior_summary,
        pn.aes("day", "incidence", group="sim")
    ) +
    pn.geom_line(alpha=0.1) +
    pn.facet_wrap("~age")
)

Evaluate the beta[0] parameters


In [ ]:
sigma_summaries = df[df["param"] == "beta[0]"]\
    .groupby("sim")\
    .aggregate({
        "value": ["mean", "median"]
    })\
    .droplevel(0, axis=1)\
    .merge(
        df_params[df_params["param"] == "beta[0]"],
        on="sim"
    )
(
    pn.ggplot(
        sigma_summaries,
        pn.aes("true", "mean")
     ) +
    pn.geom_point() +
    pn.geom_smooth(method="lm", formula="y~x")
)

## Non-centered vs centered beta parameters

Up until now, we have been looking at the non-centered beta parameters.
However, we actually care about the centered ones.


In [ ]:
sd_index = 0
for param_name, param in log_posterior._model_params.items():
    if param_name == "beta":
        break
    sd_index += len(param)

ncp_beta_sds = posteriors[:,:,sd_index, np.newaxis]
assert np.abs(np.mean(ncp_beta_sds) / 50) < 0.01
ncp_betas = posteriors[:,:,sd_index+1:sd_index+1+model.num_betas]
assert ncp_betas.shape[2] == model.num_betas
centred_betas = ncp_betas * ncp_beta_sds

In [ ]:
true_centred_betas = np.empty((len(posteriors_sims), model.num_betas))
for i, s  in enumerate(posteriors_sims):
    true_centred_betas[i] = true_params[s]["beta"][0] * np.array(true_params[s]["beta"][1:])
assert true_centred_betas.shape == (centred_betas.shape[0], centred_betas.shape[2])

In [ ]:
df_centred_betas_truth = pd.DataFrame(
    {"true": true_centred_betas.flatten()},
    index=pd.MultiIndex.from_product(
        [posteriors_sims, np.arange(model.num_betas) + 1],
        names=["sim", "index"]
    )
).reset_index()

In [ ]:
centered_beta_coverage = calc_coverage(
    truth=df_centred_betas_truth,
    posterior=centred_betas,
    remaining_axis_names=["sim", "index"],
    remaining_axis_labels=[posteriors_sims, np.arange(model.num_betas) + 1],
    groups=["index"]
)

In [ ]:
(
    pn.ggplot(centered_beta_coverage, pn.aes(colour="within")) +
    pn.geom_point(pn.aes(y="index", x="prop", xmin="ll", xmax="ul")) +
    pn.geom_errorbarh(pn.aes(y="param", xmin="ll", xmax="ul"), height = 0.001) +
    pn.facet_wrap("~interval") +
    pn.geom_vline(pn.aes(xintercept="interval/100"))
)